In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import glob

In [ ]:
# initial parameters
epochs = 100
lr = 1e-3
batch_size = 64
img_dims = (96,96,3)

data = []
labels = []

In [ ]:
# load image files from the dataset
image_files = [f for f in glob.glob('/content/drive/My Drive/Gender Detection/Dataset' + "/**/*", recursive=True) if not os.path.isdir(f)]
random.shuffle(image_files)
print(len(image_files))

2307


In [ ]:
# converting images to arrays and labelling the categories
for img in image_files:
    image = cv2.imread(img)
    image = cv2.resize(image, (img_dims[0],img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    label = img.split(os.path.sep)[-2] # Dataset\Female\face_118.jpg
    if label == "Female":
        label = 1
    else:
        label = 0
        
    labels.append([label]) # [[1], [0], [0], ...]
print(len(data))

2307


In [ ]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [ ]:
# split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2,
                                                  random_state=42)

trainY = to_categorical(trainY, num_classes=2) # [[1, 0], [0, 1], [0, 1], ...]
testY = to_categorical(testY, num_classes=2)
3+3+5

11

In [ ]:
# augmenting datset 
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")
'ppp'

'ppp'

In [ ]:

# model
def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first": #Returns a string, either 'channels_first' or 'channels_last'
        inputShape = (depth, height, width)
        chanDim = 1
        

    model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))


    #......................
    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))


    #......................
    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    #CNN ends..................
    
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes))
    model.add(Activation("sigmoid"))

    return model
'ppppp'

'ppppp'

In [ ]:

# build model
model = build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                            classes=2)
'model build done'

'model build done'

In [ ]:

# compile the model
opt = Adam(lr=lr, decay=lr/epochs)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
'model compile done'

'model compile done'

In [ ]:

# train the model
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
                        validation_data=(testX,testY),
                        steps_per_epoch=len(trainX) // batch_size,
                        epochs=epochs, verbose=1)
'model train done'

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
28/28 [==============================] - 66s 2s/step - loss: 0.8192 - accuracy: 0.6985 - val_loss: 0.6581 - val_accuracy: 0.6558
Epoch 2/100
28/28 [==============================] - 62s 2s/step - loss: 0.4966 - accuracy: 0.7900 - val_loss: 1.1680 - val_accuracy: 0.4892
Epoch 3/100
28/28 [==============================] - 62s 2s/step - loss: 0.4137 - accuracy: 0.8360 - val_loss: 0.9359 - val_accuracy: 0.5108
Epoch 4/100
28/28 [==============================] - 62s 2s/step - loss: 0.3707 - accuracy: 0.8636 - val_loss: 0.6920 - val_accuracy: 0.5476
Epoch 5/100
28/28 [==============================] - 63s 2s/step - loss: 0.3499 - accuracy: 0.8675 - val_loss: 1.0733 - val_accuracy: 0.5195
Epoch 6/100
28/28 [==============================] - 63s 2s/step - loss: 0.3249 - accuracy: 0.8827 - val_loss: 0.8805 - val_accuracy: 0.6472
Epoch 7/100
28/28 [==============================] - 63s 2s/step - loss: 0.295

'model train done'

In [ ]:
# save the model to disk
model.save('/content/drive/My Drive/Gender Detection/gd.model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/Gender Detection/gd.model/assets


In [ ]:

# plot training/validation loss/accuracy
plt.style.use("ggplot")
plt.figure()
N = epochs
plt.plot(np.arange(0,N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0,N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0,N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0,N), H.history["val_acc"], label="val_acc")

plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")


In [ ]:
# save plot to disk
plt.savefig('plot.png')

<Figure size 432x288 with 0 Axes>